In [2]:
import pandas as pd
import importlib
import berechnen_wp, pv_profil, lastprofile_VDI4655, temperatur_aussen, try_region, heizkurve
importlib.reload(berechnen_wp)

h, w, twe, s = lastprofile_VDI4655.get_jahresenergiebedarf('1984 - 1994', 200, 3, 0)
TRY_region, T_n_aussen = try_region.get_try_t_n_aussen(40599)
df = lastprofile_VDI4655.get_lastprofile(w, s, twe, 200, TRY_region, 3)
df['T_aussen'] = temperatur_aussen.get_hourly_temperature(40599, 2014)
hz, T_soll, T_n_vor, T_n_rueck = heizkurve.get_heizkurve('Heizkörper', df['T_aussen'], T_n_aussen)
df['T_vor'] = hz['T_vor']
df['T_rueck'] = hz['T_rueck']
wp_groesse = berechnen_wp.get_waermepumpe(h)
heizleistung_auslegung = heizkurve.get_heizleistung(T_n_aussen, h, T_soll)
df = heizkurve.get_heizleistung_profil(df, heizleistung_auslegung)
df = heizkurve.get_cop(wp_groesse, df)
V_ps, PS_verlust, Q_ps = berechnen_wp.get_pufferspeicher(h, T_n_vor, T_n_rueck)
df, P_el, COP = berechnen_wp.ohne_pv(df, Q_ps, PS_verlust)
pv = pv_profil.get_pv_profil(40599, 2014, 10)
df_pv = berechnen_wp.mit_pv(df.copy(), pv)
df_pvbs = berechnen_wp.mit_pvbs(df.copy(), pv, 10, 11)

The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




In [3]:
importlib.reload(berechnen_wp)
strompreis = 0.358
ergebnisse_pv = berechnen_wp.ersparnis_pv(df_pv, 10, strompreis)
berechnen_wp.print_ersparnis(ergebnisse_pv)

Haushaltsstrombedarf in kWh: 4980
Wärmepumpe Strombedarf in kWh: 6634
Jahresertrag in kWh: 11282
Eigenverbrauch in kWh: 3676
Geladene PV-Strom in Wärmepumpe in kWh: 1714

Netzbezug in kWh: 6224
Einspeisung ins Netz in kWh: 7606

Stromkosten ohne PV in €/a: 4157.82
Stromkosten mit PV in €/a: 2228.19
Einspeisevergütung in €/a: 605.44
Stromkosten Einsparung in €/a: 2535.07


In [4]:
importlib.reload(berechnen_wp)
ergebnisse_bs = berechnen_wp.ersparnis_bs(df_pvbs, 10, strompreis)
berechnen_wp.print_ersparnis(ergebnisse_bs)

Haushaltsstrombedarf in kWh: 4980
Wärmepumpe Strombedarf in kWh: 6634
Jahresertrag in kWh: 11282
Eigenverbrauch in kWh: 5390
Geladene PV-Strom in Wärmepumpe in kWh: 2135
Geladene PV-Strom in Batteriespeicher in kWh: 2098
Geladene BS-Strom in Wärmepumoe in kWh: 591

Netzbezug in kWh: 5920
Einspeisung ins Netz in kWh: 5421

Stromkosten ohne PV in €/a: 4157.82
Stromkosten mit PV & BS in €/a: 2119.36
Einspeisevergütung in €/a: 431.51
Stromkosten Einsparung in €/a: 2469.97


In [5]:
importlib.reload(berechnen_wp)
df_ev = berechnen_wp.mit_pvev(df.copy(), pv, True)

In [11]:
importlib.reload(berechnen_wp)
df_pvbsev = berechnen_wp.mit_pvbsev(df.copy(), pv, 10, 11, True)

In [12]:
# Finden der Indizes, wo der Wert in df['überschuss'] negativ ist
negative_indices = df_pvbsev[df_pvbsev['überschuss'] < 0].index

# Ausgabe der Indizes
print("Indizes mit negativem Überschuss:")
print(negative_indices)

Indizes mit negativem Überschuss:
DatetimeIndex([], dtype='datetime64[ns]', name='Zeit', freq='h')


In [14]:
# Gebe den Index der gewünschten Zeile an
i = '2014-01-18 09:00:00'  # Beispiel: Zeile mit Index 10
row = df_pvbsev.loc[i]  # Hole die Zeile als Series

# Hole die Werte aus der Zeile
pv_ertrag = row['PV Ertrag']
strombedarf = row['Strombedarf']
p_el_wp = row['elekt. Leistungaufnahme']
charge_to_battery = row['battery_charge'] if 'battery_charge' in row else 0
charge_efficiency = 0.96  # Beispielwert
max_soc = 11  # Maximale Speicherkapazität der Batterie in kWh
battery_soc = row['battery_soc'] if 'battery_soc' in row else 5  # Initialwert der Batterie

# Debugging-Berechnung für diese Zeile
eigenverbrauch_str = 0
eigenverbrauch_wp = 0
eigenverbrauch_battery = 0
ueberschuss = 0

# Debug: Eingabewerte anzeigen
print(f"Index {i} - Eingabewerte:")
print(f"pv_ertrag: {pv_ertrag}, strombedarf: {strombedarf}, p_el_wp: {p_el_wp}, charge_to_battery: {charge_to_battery}")

# Step 1: Eigenverbrauch für Strombedarf (Prio 1)
if pv_ertrag >= strombedarf:
    eigenverbrauch_str = strombedarf
    ueberschuss = pv_ertrag - strombedarf
else:
    eigenverbrauch_str = pv_ertrag
    ueberschuss = 0

print(f"Nach Strombedarf: eigenverbrauch_str = {eigenverbrauch_str}, ueberschuss = {ueberschuss}")

# Step 2: Eigenverbrauch für WP (Prio 2)
if ueberschuss > 0 and p_el_wp > 0:
    if ueberschuss >= p_el_wp:
        eigenverbrauch_wp = p_el_wp
        ueberschuss -= p_el_wp
    else:
        eigenverbrauch_wp = ueberschuss
        ueberschuss = 0

print(f"Nach WP: eigenverbrauch_wp = {eigenverbrauch_wp}, ueberschuss = {ueberschuss}")

# Step 3: Eigenverbrauch für Batterie (Prio 3)
if ueberschuss > 0:
    charge_potential = ueberschuss * charge_efficiency
    charge_to_battery = min(charge_potential, max_soc - battery_soc)
    eigenverbrauch_battery = charge_to_battery / charge_efficiency
    ueberschuss -= eigenverbrauch_battery

print(f"Nach Batterie: eigenverbrauch_battery = {eigenverbrauch_battery}, ueberschuss = {ueberschuss}")

# Gesamter Eigenverbrauch
eigenverbrauch = eigenverbrauch_str + eigenverbrauch_wp + eigenverbrauch_battery

print(f"Gesamter Eigenverbrauch: {eigenverbrauch}")


Index 2014-01-18 09:00:00 - Eingabewerte:
pv_ertrag: 3.5189, strombedarf: 0.2641600214825872, p_el_wp: 1.8948212235445725, charge_to_battery: 1.3055220047739267
Nach Strombedarf: eigenverbrauch_str = 0.2641600214825872, ueberschuss = 3.2547399785174127
Nach WP: eigenverbrauch_wp = 1.8948212235445725, ueberschuss = 1.3599187549728402
Nach Batterie: eigenverbrauch_battery = 1.3599187549728404, ueberschuss = -2.220446049250313e-16
Gesamter Eigenverbrauch: 3.5189000000000004
